# 작업형  2유형 - 예측

## 성능이 우수한 예측 모형을 구축하려면 다음 작업이 필요하다.
* 적절한 데이터 전처리
* Feature Engineering
* 분류 알고리즘 사용
* 초매개변수(하이퍼파라미터) 최적화
* 모형 앙상블

### 고객 3,500명(x_train, y_train)에 대한 성별 예측 모형을 만든 후 평가용 데이터(x_test)에 적용하여 2,482명의 성별 예측값(남자일 확률?)을 CSV 파일로 생성하기

* 제출한 모델의 성능은 ROC-AUC 평가지표에 따라 채점

#### 기초 정보 확인하기

In [1]:
import pandas as pd

x_train = pd.read_csv('x_train.csv', encoding='cp949')
x_test = pd.read_csv('x_test.csv', encoding='cp949')
y_train = pd.read_csv('y_train.csv', encoding='cp949')

# print(x_train.columns)
x_train.info()
x_train.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3500 entries, 0 to 3499
Data columns (total 10 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   cust_id  3500 non-null   int64  
 1   총구매액     3500 non-null   int64  
 2   최대구매액    3500 non-null   int64  
 3   환불금액     1205 non-null   float64
 4   주구매상품    3500 non-null   object 
 5   주구매지점    3500 non-null   object 
 6   내점일수     3500 non-null   int64  
 7   내점당구매건수  3500 non-null   float64
 8   주말방문비율   3500 non-null   float64
 9   구매주기     3500 non-null   int64  
dtypes: float64(3), int64(5), object(2)
memory usage: 273.6+ KB


,cust_id,총구매액,최대구매액,환불금액,내점일수,내점당구매건수,주말방문비율,구매주기
count,3500.000000,3.500000e+03,3.500000e+03,1.205000e+03,3500.000000,3500.000000,3500.000000,3500.000000
mean,1749.500000,9.191925e+07,1.966424e+07,2.407822e+07,19.253714,2.834963,0.307246,20.958286
std,1010.507298,1.635065e+08,3.199235e+07,4.746453e+07,27.174942,1.912368,0.289752,24.748682
min,0.000000,-5.242152e+07,-2.992000e+06,5.600000e+03,1.000000,1.000000,0.000000,0.000000
25%,874.750000,4.747050e+06,2.875000e+06,2.259000e+06,2.000000,1.666667,0.027291,4.000000
50%,1749.500000,2.822270e+07,9.837000e+06,7.392000e+06,8.000000,2.333333,0.256410,13.000000
75%,2624.250000,1.065079e+08,2.296250e+07,2.412000e+07,25.000000,3.375000,0.448980,28.000000
max,3499.000000,2.323180e+09,7.066290e+08,5.637530e+08,285.000000,22.083333,1.000000,166.000000


#### 전처리

In [2]:
# cust_id는 분석 과정에서 불필요하지만 마지막에 CSV 파일 만들 때 필요하므로
# 따로 남겨놓기
x_train_cust_id = x_train.pop('cust_id')
y_train_cust_id = y_train.pop('cust_id')
x_test_cust_id = x_test.pop('cust_id')

In [3]:
# 결측치 확인 후 환불금액만 결측치가 있음
# 이 결측치는 환불을 하지 않은 것이므로 0으로 채움
print(x_train.info())
x_train['환불금액'] = x_train['환불금액'].fillna(0)
print(x_train.info())

# 이거 x_test도 해줘야함 나중에 모델 사용할 때 결측치 있으면 안됨
x_test['환불금액'] = x_test['환불금액'].fillna(0)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3500 entries, 0 to 3499
Data columns (total 9 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   총구매액     3500 non-null   int64  
 1   최대구매액    3500 non-null   int64  
 2   환불금액     1205 non-null   float64
 3   주구매상품    3500 non-null   object 
 4   주구매지점    3500 non-null   object 
 5   내점일수     3500 non-null   int64  
 6   내점당구매건수  3500 non-null   float64
 7   주말방문비율   3500 non-null   float64
 8   구매주기     3500 non-null   int64  
dtypes: float64(3), int64(4), object(2)
memory usage: 246.2+ KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3500 entries, 0 to 3499
Data columns (total 9 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   총구매액     3500 non-null   int64  
 1   최대구매액    3500 non-null   int64  
 2   환불금액     3500 non-null   float64
 3   주구매상품    3500 non-null   object 
 4   주구매지점    3500 non-null   object 
 5   내점일수     3500 non-null   int64

In [4]:
# 범주형 변수 인코딩(주구매상품, 주구매지점)
# 주구매상품 unique 값 확인
print(x_train['주구매상품'].unique())
print(x_train['주구매지점'].unique())

# 이후 레이블 인코딩하기
# unique한 값이 많아 원핫인코딩하면 컬럼이 너무 많이 생성됨
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()

x_train['주구매상품'] = encoder.fit_transform(x_train['주구매상품'])
# 인코딩 변환 순서 확인
# print(encoder.classes_)
x_train['주구매지점'] = encoder.fit_transform(x_train['주구매지점'])
# 인코딩 변환 순서 확인
# print(encoder.classes_)

# x_train을 인코딩하면 x_test도 인코딩해줘야함
x_test['주구매상품'] = encoder.fit_transform(x_test['주구매상품'])
# 인코딩 변환 순서 확인
# print(encoder.classes_)
x_test['주구매지점'] = encoder.fit_transform(x_test['주구매지점'])
# 인코딩 변환 순서 확인
# print(encoder.classes_)

['기타' '스포츠' '남성 캐주얼' '보석' '디자이너' '시티웨어' '명품' '농산물' '화장품' '골프' '구두' '가공식품'
 '수산품' '아동' '차/커피' '캐주얼' '섬유잡화' '육류' '축산가공' '젓갈/반찬' '액세서리' '피혁잡화' '일용잡화'
 '주방가전' '주방용품' '건강식품' '가구' '주류' '모피/피혁' '남성 트랜디' '셔츠' '남성정장' '생활잡화'
 '트래디셔널' '란제리/내의' '커리어' '침구/수예' '대형가전' '통신/컴퓨터' '식기' '소형가전' '악기']
['강남점' '잠실점' '관악점' '광주점' '본  점' '일산점' '대전점' '부산본점' '분당점' '영등포점' '미아점'
 '청량리점' '안양점' '부평점' '동래점' '포항점' '노원점' '창원점' '센텀시티점' '인천점' '대구점' '전주점'
 '울산점' '상인점']


/Users/dmis/opt/anaconda3/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [5]:
# 파생변수 만들기
# 환불여부(환불금액이 0보다 크면 1, 0이면 0)
x_train['환불여부'] = x_train['환불금액'] >0 
x_train.astype(int)

# 환불여부 만들었으니 기존 환불금액은 삭제하기
x_train = x_train.drop(columns = '환불금액')
x_train

# 위의 과정 x_test도 해줘야함 안그러면 안됨
x_test['환불여부'] = x_test['환불금액'] >0
x_test.astype(int)
x_test = x_test.drop(columns = '환불금액')

In [6]:
# 데이터 스케일링
# 연속형 변수 표준화하기
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

# 근데 레이블인코딩한 애들은 표준화하면 좀 그렇지 않나? 이상하네
x_train = pd.DataFrame(scaler.fit_transform(x_train), columns = x_train.columns)
print(x_train.describe())

# x_train만 표준화하고 x_test는 표준화하지 않는 것 조심: 둘 다 표준화 해야 함
x_test = pd.DataFrame(scaler.fit_transform(x_test), columns = x_test.columns)
print(x_test.describe())

               총구매액         최대구매액         주구매상품         주구매지점          내점일수  \
count  3.500000e+03  3.500000e+03  3.500000e+03  3.500000e+03  3.500000e+03   
mean  -3.349701e-17 -4.263256e-17 -6.090366e-17 -8.767590e-17  4.669281e-17   
std    1.000143e+00  1.000143e+00  1.000143e+00  1.000143e+00  1.000143e+00   
min   -8.829091e-01 -7.082781e-01 -1.122438e+00 -1.904703e+00 -6.718072e-01   
25%   -5.332182e-01 -5.248643e-01 -7.383573e-01 -4.851745e-01 -6.350034e-01   
50%   -3.896215e-01 -3.072187e-01 -4.310925e-01 -3.077335e-01 -4.141802e-01   
75%    8.923657e-02  1.031100e-01  5.675181e-01  7.569129e-01  2.114855e-01   
max    1.364826e+01  2.147585e+01  2.027026e+00  2.176441e+00  9.780490e+00   

            내점당구매건수        주말방문비율          구매주기          환불여부  
count  3.500000e+03  3.500000e+03  3.500000e+03  3.500000e+03  
mean  -2.618858e-16  7.917476e-17 -4.263256e-17  1.218073e-17  
std    1.000143e+00  1.000143e+00  1.000143e+00  1.000143e+00  
min   -9.596611e-01 -1.060530e+0

In [7]:
# 피쳐간 상관관계확인 후 처리
print(x_train.corr())
# 상관계수의 절댓값이 0.6 이상이면 강한 상관관계가 존재한다고 판단함
# 따라서 회귀 시 다중공선성 우려로 둘 중 하나 삭제 필요
# 여기서는 '최대구매액'을 삭제
# 이때 x_train, x_test 모두 삭제해줘야 함

x_train.drop(columns = '최대구매액', inplace = True)
x_test.drop(columns = '최대구매액', inplace = True)

print(x_train.columns)

             총구매액     최대구매액     주구매상품     주구매지점      내점일수   내점당구매건수    주말방문비율  \
총구매액     1.000000  0.700080 -0.082916 -0.038724  0.659084  0.090022  0.014396   
최대구매액    0.700080  1.000000 -0.090729 -0.024819  0.374147  0.018980  0.022277   
주구매상품   -0.082916 -0.090729  1.000000  0.015874 -0.185275 -0.274178 -0.010018   
주구매지점   -0.038724 -0.024819  0.015874  1.000000 -0.059570 -0.080804  0.011990   
내점일수     0.659084  0.374147 -0.185275 -0.059570  1.000000  0.225264 -0.010325   
내점당구매건수  0.090022  0.018980 -0.274178 -0.080804  0.225264  1.000000  0.007659   
주말방문비율   0.014396  0.022277 -0.010018  0.011990 -0.010325  0.007659  1.000000   
구매주기    -0.212944 -0.115837  0.032469  0.035344 -0.293200 -0.091151  0.003372   
환불여부     0.403357  0.330687 -0.121414 -0.061974  0.498958  0.063000 -0.008775   

             구매주기      환불여부  
총구매액    -0.212944  0.403357  
최대구매액   -0.115837  0.330687  
주구매상품    0.032469 -0.121414  
주구매지점    0.035344 -0.061974  
내점일수    -0.293200  0.498958  
내점당구매건수 -

#### 학습하고 예측하기

In [8]:
# 데이터 학습시키기
# 성별(male or female)은 이진 분류임
# 여기서는 기본적인 의사결정나무 분류기(DecisionTreeclassifier) 사용
# 이때 predict()로 나오는 결과는 Series이므로 pd.DataFrame 처리가 필요

# 패키지 import 하기 어려울 때
# import sklearn
# print(dir(sklearn)) <- 여기서 tree 찾기
# import sklearn.tree
# print(help(sklearn.tree)) <- 여기서 DecisionTreeClassifier 찾기

from sklearn.tree import DecisionTreeClassifier

# model 객체 만들기
model = DecisionTreeClassifier()

# x_train, y_train으로 학습시키기
model.fit(x_train, y_train)

DecisionTreeClassifier()

In [9]:
# 학습된 모델을 이용하여 x_test의 종속변수 y_test_predict만들기
y_test_predict = model.predict(x_test)
y_test_predict = pd.DataFrame(y_test_predict)
y_test_predict

,0
0,1
1,0
2,0
3,0
4,0
...,...
2477,1
2478,0
2479,1
2480,1


In [10]:
# 하이퍼파라미터 튜닝하기
# 의사결정나무 분류기에서 대표적으로 사용하는 하이퍼파라미터 : max_depth, criterion
model_with_hyperparam = DecisionTreeClassifier(max_depth = 10, 
                                               criterion = 'entropy')

# 새로운 모델로 학습하기
model_with_hyperparam.fit(x_train, y_train)
y_test_predict_with_hyperparam = model_with_hyperparam.predict(x_test)
y_test_predict_with_hyperparam = pd.DataFrame(y_test_predict_with_hyperparam)
y_test_predict_with_hyperparam

,0
0,1
1,0
2,0
3,0
4,0
...,...
2477,0
2478,0
2479,1
2480,1


In [11]:
# 결과 예측하기
y_test_proba = model_with_hyperparam.predict_proba(x_test)
y_test_proba = pd.DataFrame(y_test_proba)
# y_test_proba.head(5)

# 최종적으로 남성(1)로 분류될 확률을 구해야함
result = y_test_proba[1]
result = pd.DataFrame(result)

#### 평가하기

In [12]:
# y_test의 실제값이 없으므로 x_train으로 y_train 예측 평가하기
y_train_predict = model_with_hyperparam.predict(x_train)

# roc평가지표 계산하기 위한 함수 가져오기
from sklearn.metrics import roc_auc_score

# roc_auc_score 확인하기
print(roc_auc_score(y_train, y_train_predict))

0.6894922453433092


#### 결과 제출하기

In [13]:
# x_test_cust_id랑 result 합치기
# 이때 열을 합치는 것이므로 axis = 1
# 행을 합치는 것이면 axis = 0
df = pd.concat([x_test_cust_id, y_test_predict_with_hyperparam], axis = 1)
df.columns = ['cust_id', 'gender']
print(df)

# CSV 파일로 저장하기
df.to_csv('12345.csv', index = False)
df1 = pd.read_csv('12345.csv')

      cust_id  gender
0        3500       1
1        3501       0
2        3502       0
3        3503       0
4        3504       0
...       ...     ...
2477     5977       0
2478     5978       0
2479     5979       1
2480     5980       1
2481     5981       1

[2482 rows x 2 columns]


### titanic 데이터로 생존 여부 예측하기

* 제출한 모델의 성능은 ROC-AUC 평가지표에 따라 채점

#### 기초 정보 확인하기

In [14]:
import pandas as pd
x_train = pd.read_csv('titanic_x_train.csv', encoding='cp949')
x_test = pd.read_csv('titanic_x_test.csv', encoding='cp949')
y_train = pd.read_csv('titanic_y_train.csv', encoding='cp949')

x_train.info()
y_train.columns = ['PassengerId', 'Survived']
y_train.info()
x_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   티켓등급         891 non-null    int64  
 2   승객이름         891 non-null    object 
 3   성별           891 non-null    object 
 4   나이           714 non-null    float64
 5   형제자매배우자수     891 non-null    int64  
 6   부모자식수        891 non-null    int64  
 7   티켓번호         891 non-null    object 
 8   운임요금         891 non-null    float64
 9   객실번호         204 non-null    object 
 10  선착장          889 non-null    object 
dtypes: float64(2), int64(4), object(5)
memory usage: 76.7+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype
---  ------       --------------  -----
 0   PassengerId  891 non-null    int64
 1   Survived     891 non-null    int64
dtypes: int64(2)
memor

In [15]:
# 필요없어 보이는 객실번호, 승객이름 삭제하기
x_train.drop(columns = ['객실번호', '승객이름', '티켓번호'], inplace = True)
x_test.drop(columns = ['객실번호', '승객이름', '티켓번호'], inplace = True)

In [16]:
# 예측에 필요하지 않지만 이후 필요한 PassengerId 따로 저장해놓기
x_train_passenger_id = x_train.pop('PassengerId')
y_train_passenger_id = y_train.pop('PassengerId')
x_test_passenger_id = x_test.pop('PassengerId')

In [17]:
# 이후 데이터 확인
x_train.info()
x_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   티켓등급      891 non-null    int64  
 1   성별        891 non-null    object 
 2   나이        714 non-null    float64
 3   형제자매배우자수  891 non-null    int64  
 4   부모자식수     891 non-null    int64  
 5   운임요금      891 non-null    float64
 6   선착장       889 non-null    object 
dtypes: float64(2), int64(3), object(2)
memory usage: 48.9+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   티켓등급      418 non-null    int64  
 1   성별        418 non-null    object 
 2   나이        332 non-null    float64
 3   형제자매배우자수  418 non-null    int64  
 4   부모자식수     418 non-null    int64  
 5   운임요금      417 non-null    float64
 6   선착장       418 non-null    object 
dtypes: float64(2), int64(3

In [18]:
# 원핫인코딩을 해도 될지 티켓등급, 성별 unique한 값 확인
print(x_train['티켓등급'].unique())
print(x_train['성별'].unique())
print(x_train['선착장'].unique())

# 선착장 결측치 최빈값으로 대체하기
marina_mode = x_train['선착장'].mode()
x_train['선착장'] = x_train['선착장'].fillna(marina_mode[0])
x_train.info()

[3 1 2]
['male' 'female']
['S' 'C' 'Q' nan]
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   티켓등급      891 non-null    int64  
 1   성별        891 non-null    object 
 2   나이        714 non-null    float64
 3   형제자매배우자수  891 non-null    int64  
 4   부모자식수     891 non-null    int64  
 5   운임요금      891 non-null    float64
 6   선착장       891 non-null    object 
dtypes: float64(2), int64(3), object(2)
memory usage: 48.9+ KB


In [19]:
# 원핫인코딩할까하다가 선착장때문에 그냥 레이블인코딩하기
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
x_train['성별'] = encoder.fit_transform(x_train['성별'])
x_test['성별'] = encoder.fit_transform(x_test['성별'])
x_train['선착장'] = encoder.fit_transform(x_train['선착장'])
x_test['선착장'] = encoder.fit_transform(x_test['선착장'])

In [25]:
# 나이 결측치는 중앙값으로 대체하기
x_train.info()
x_train.describe()

age_median = x_train['나이'].median()
age_median
x_train['나이'].fillna(age_median, inplace = True)
x_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   티켓등급      891 non-null    int64  
 1   성별        891 non-null    int64  
 2   나이        714 non-null    float64
 3   형제자매배우자수  891 non-null    int64  
 4   부모자식수     891 non-null    int64  
 5   운임요금      891 non-null    float64
 6   선착장       891 non-null    int64  
dtypes: float64(2), int64(5)
memory usage: 48.9 KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   티켓등급      891 non-null    int64  
 1   성별        891 non-null    int64  
 2   나이        891 non-null    float64
 3   형제자매배우자수  891 non-null    int64  
 4   부모자식수     891 non-null    int64  
 5   운임요금      891 non-null    float64
 6   선착장       891 non-null    int64  
dtypes: float64(2), int64(5)
memory usa

In [36]:
# x_test도 같은 방법으로 전처리해주기
x_test.info()

# 나이, 운임요금을 모두 중앙값으로 대체하기
test_age_med = x_test['나이'].median()
test_fee_med = x_test['운임요금'].median()

x_test['나이'].fillna(test_age_med, inplace = True)
x_test['운임요금'].fillna(test_fee_med, inplace = True)
x_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   티켓등급      418 non-null    int64  
 1   성별        418 non-null    int64  
 2   나이        332 non-null    float64
 3   형제자매배우자수  418 non-null    int64  
 4   부모자식수     418 non-null    int64  
 5   운임요금      417 non-null    float64
 6   선착장       418 non-null    int64  
dtypes: float64(2), int64(5)
memory usage: 23.0 KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   티켓등급      418 non-null    int64  
 1   성별        418 non-null    int64  
 2   나이        418 non-null    float64
 3   형제자매배우자수  418 non-null    int64  
 4   부모자식수     418 non-null    int64  
 5   운임요금      418 non-null    float64
 6   선착장       418 non-null    int64  
dtypes: float64(2), int64(5)
memory usa

In [44]:
# 예측모델만들기
from sklearn.tree import DecisionTreeClassifier 
model = DecisionTreeClassifier(max_depth = 10, criterion = 'entropy')
model.fit(x_train, y_train)

# x_train으로 검증해보기
# 원래는 train 데이터의 80%를 학습용, 20%를 검증용으로 하는데 처음이니까 그냥 100% 다 해보기
train_predict = pd.DataFrame(model.predict(x_train))

# roc평가지표 계산하기 위한 함수 가져오기
from sklearn.metrics import roc_auc_score

# roc_auc_score 확인하기 -> 0.89
print(roc_auc_score(y_train, train_predict))

0.8920285686894833


In [50]:
# 예측 모델에 x_test 집어넣고 CSV 파일 만들기
test_predict = pd.DataFrame(model.predict(x_test))
result = pd.concat([x_test_passenger_id, test_predict], axis = 1)
result.columns = ['PassenggerId', 'Survived']
print(result)

result.to_csv('result.csv', index = False)

result.shape

     PassenggerId  Survived
0             892         0
1             893         0
2             894         0
3             895         0
4             896         1
..            ...       ...
413          1305         0
414          1306         1
415          1307         0
416          1308         0
417          1309         0

[418 rows x 2 columns]


(418, 2)

### 위에서  보완할 점
* 전처리 중 스케일링이 되지 않았다.
    * StandardScaling 또는 RobustScaling을 했어야 한다.
* 검증데이터(data for validation)를 만들고 검증을 하지 않았다.
    * 실제 시험에서는 검증데이터가 없으면 모델의 성능이 얼마나 되는지 알 수가 없다.

#### 보완해보기 <- 처음부터 다시

In [89]:
import pandas as pd
x_train = pd.read_csv('titanic_x_train.csv', encoding='cp949')
x_test = pd.read_csv('titanic_x_test.csv', encoding='cp949')
y_train = pd.read_csv('titanic_y_train.csv', encoding='cp949')

# x_train.info()
y_train.columns = ['PassengerId', 'Survived']
# y_train.info()
# x_test.info()

x_train.drop(columns = ['PassengerId'], inplace = True)
y_train.drop(columns = ['PassengerId'], inplace = True)
x_test_PassengerId = x_test.pop('PassengerId')

# print(x_train.isnull().sum(), x_test.isnull().sum())

# 나이 중앙값으로 채워주기
x_train_age_med = x_train['나이'].median()
x_test_age_med = x_test['나이'].median()

x_train['나이'].fillna(x_train_age_med, inplace = True)
x_test['나이'].fillna(x_test_age_med, inplace = True)
# print(x_train.isnull().sum(), x_test.isnull().sum())

# 선착장 최빈값으로 채워주기
x_train_marina_mode = x_train['선착장'].mode()
x_test_marina_mode = x_test['선착장'].mode()
# print(x_train_marina_mode[0], x_test_marina_mode[0])

x_train['선착장'].fillna(x_train_marina_mode[0], inplace = True)
x_test['선착장'].fillna(x_test_marina_mode[0], inplace = True)
# print(x_train.isnull().sum(), x_test.isnull().sum())

# test 데이터의 운임요금 중앙값으로 대체하기
x_test_fee_med = x_test['운임요금'].median()
x_test['운임요금'].fillna(x_test_fee_med, inplace = True)
# print(x_train.isnull().sum(), x_test.isnull().sum())

# 형제자매배우자수, 부모자식수를 합하여 가족수라는 파생변수 만들기
x_train['가족수'] = x_train['형제자매배우자수'] + x_train['부모자식수']
x_test['가족수'] = x_test['형제자매배우자수'] + x_test['부모자식수']

# 필요하지 않은 피처(형제자매배우자수, 부모자식수, 티켓번호, 객실번호, 승객이름) 지우기
x_train.drop(columns = ['형제자매배우자수', '부모자식수', '티켓번호', '객실번호', '승객이름'], inplace = True)
x_test.drop(columns = ['형제자매배우자수', '부모자식수', '티켓번호', '객실번호', '승객이름'], inplace = True)

# 뭔과 관련있지 않을까 해서 확인
# x_train[['티켓등급', '운임요금']].corr()

# 성별, 선착장 레이블인코딩하기
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()

x_train['성별'] = encoder.fit_transform(x_train['성별'])
x_train['선착장'] = encoder.fit_transform(x_train['선착장'])
x_test['성별'] = encoder.fit_transform(x_test['성별'])
x_test['선착장'] = encoder.fit_transform(x_test['선착장'])
# print(x_train.info(), x_test.info())

# Robust Scaling 하기
from sklearn.preprocessing import RobustScaler
scaler = RobustScaler()

x_train = pd.DataFrame(scaler.fit_transform(x_train), columns = x_train.columns)
x_test = pd.DataFrame(scaler.fit_transform(x_test), columns = x_test.columns)

# 훈련데이터와 검증데이터 나누기
# 보통 훈련데이터와 검증데이터의 비율은 8:2정도로 한다.
from sklearn.model_selection import train_test_split
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size = 0.2)

# 훈련 후 검증하기
from sklearn.tree import DecisionTreeClassifier
model = DecisionTreeClassifier(max_depth = 10, criterion = 'gini')
model.fit(x_train, y_train)

y_val_predict = pd.DataFrame(model.predict(x_val))
y_val_predict.columns = ['Survived']

# 검증은 roc_auc_score로 해보기
from sklearn.metrics import roc_auc_score, f1_score
# print(roc_auc_score(y_val, y_val_predict))
# max_depth = 10, criterion = 'entropy' -> score : 0.8076765188834154
# max_depth = 10, criterion = 'gini' -> score : 0.8407557354925775

# 제출할 자료(y_test)만들기
y_test = pd.DataFrame(model.predict(x_test))
y_test.columns = ['Survived']

result = pd.concat([x_test_PassengerId, y_test], axis = 1)
result.to_csv('Submission.csv', index = False)